In [1]:
# jupyter notebook --config=/home/nakamura/.jupyter/jupyter_notebook_config.py
# %matplotlib nbagg
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import time
sys.path.append("..")
os.chdir("/home/nakamura/HAI/main_server/hai")

import coloredlogs, logging
logger = logging.getLogger(__name__)
coloredlogs.install(level='CRITICAL', logger=logger)

import pymongo
from pymongo import MongoClient
from PIL import Image
from matplotlib import pylab as plt
from matplotlib.animation import ArtistAnimation
import matplotlib.animation as animation
import numpy as np
from bson.objectid import ObjectId
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from notebooks.utils.utils import visualize, display_latest_image, display_image, print_time, strtime_to_epoch, epoch_to_strtime, UpdateDist, display_two_images
from controllers.actionrec import ActionRecognition
from controllers.dbreader.hue_koki_dbreader import HueDBReader, pair_images, extract_color, ExprHueDBReader
from controllers.dbreader.opedbreader import OperationDBReader, collect_img_op
from controllers.dbreader.imagereader import ImageReader
from controllers.vectorizer.posennvectorizer import PoseNNVectorizer
from controllers.vectorizer.person2vec import Person2Vec

mongo = MongoClient('localhost', 20202).hai

from flask import Flask
app = Flask(__name__)
app.config.from_pyfile(filename="application.cfg")

/home/nakamura/.pyenv/versions/anaconda3-4.4.0/envs/detection_server/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


True

In [ ]:
from controllers.learner.i3dnn import I3DNN
from .controller import Controller
from controllers.utils import filter_persons
import cv2
import numpy as np
import time
class ActionRecognition(Controller):
    def __init__(self):
        self.nn = I3DNN()

    def on_event(self, event, data):
        if event == "image":
            if app.config['ENCRYPTION']:
                image_path = app.config['ENCRYPTED_IMG_DIR'] + data['filename']
            else:
                image_path = app.config['RAW_IMG_DIR'] + data['filename']

            n = mongo.images.find_one({"_id": data["_id"]})
            try:
                persons, pose_indices = filter_persons(n)
            except:
                return
            
            img = cv2.imread(image_path)
            whole_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 128.0 - 1.0
            updates = {"history.action_request": time.time()}
            
            for i in range(len(n["detections"])):
                if n["detections"][i]["label"] == "person":
                    updates["detections.{}.passed".format(i)] = i in persons
                    if i in persons:
                        box = n["detections"][i]["box"]
                        longer_side = max((box[2]-box[0])*2.0,(box[3]-box[1])*2.0)
                        longer_side = min(min(whole_img.shape[1], longer_side), whole_img.shape[0])
                            
                        a = int(longer_side/2)
                        cx, cy = (box[0]+box[2])//2, (box[1]+box[3])//2
                        x1, y1, x2, y2 = cx-a, cy-a, cx+a, cy+a
                        if x1 < 0:
                            x2 -= x1
                            x1 = 0
                        if y1 < 0:
                            y2 -= y1
                            y1 = 0
                        if x2 >= whole_img.shape[1]:
                            x1 -= x2-whole_img.shape[1]
                            x2 = whole_img.shape[1]
                        if y2 >= whole_img.shape[0]:
                            y1 -= y2-whole_img.shape[0]
                            y2 = whole_img.shape[0]
                            
                        crop = whole_img[y1:y2,x1:x2,:]
                        crop = cv2.resize(crop, (224, 224))
                        img = np.array([[crop for _ in range(10)]])
                        prob, logits, label, feats = self.nn.process_image(img)
                        updates["detections.{}.action".format(i)] = label
                        updates["detections.{}.action_confidence".format(i)] = float(prob)
                        updates["detections.{}.action_crop".format(i)] = [x1,y1,x2,y2]
                        updates["detections.{}.action_vector".format(i)] = feats
                        
                        a = persons.index(i)
                        if pose_indices[a] is not None:
                            updates["detections.{}.pose_body_index".format(i)] = pose_indices[a]

            updates["history.action_recorded"] = time.time()
            mongo.images.update_one({"_id": data["_id"]}, {'$set': updates}, upsert=False)

In [2]:
action_rec = ActionRecognition()

INFO:tensorflow:Restoring parameters from ../../../../sean/kinetics-i3d/data/checkpoints/rgb_imagenet/model.ckpt


In [14]:
start = strtime_to_epoch("2017-12-19 23:45:00")
end = start + 30 
# start = strtime_to_epoch("2017-12-19 23:45:00")
# end = time.time()

In [15]:
imgs_cols = mongo.images.find({'time': {'$gt': start, '$lt': end}})

In [16]:
imgs_cols.count()

14

In [17]:
start

1513694700.0

In [ ]:
for img_col in imgs_cols:
    action_rec.on_event('image', )